# CVXOPT

- [Official Documentation](https://cvxopt.org/userguide/index.html)

In CVXOPT the main thing that we need to do is write the cost function in  either of the following forms: Linear Programming, Quadratic Programming, Second-Order Cone Programming, Semidefinite Programming. 

For our work we are using a Quadratic solver so we are writing our cost function $minimize \mid\mid X_{curr} - X_{goal} \mid \mid_2$ in the quadratic form as:

$$\begin{array}{ll}
\mbox{minimize}   & (1/2) x^TPx + q^T x \\
\mbox{subject to} & G x \preceq h \\
                  & Ax = b.
\end{array}$$

The derivation of the quadratic form from the cost function is something we need to do by hand and in the quadratic form, we need to only find the P, q, G, h ,A and b matrices.  Once we find these matrices, we then need to create these cvxopt matrices and input them in the solver as ```cvxopt.solvers.qp(P, q, G, h, A, b)``` and we will get our solution if it exists. 

Below is a simple example:

In [1]:
import numpy as np
import cvxopt as cvx

## CVXOPT matrices

The cvxopt solver will only accept cvxopt matrices. We can create a cvxopt matrix as ```cvxopt.matrix(x,size,tc)```.

Here 
- x is a a number or a sequence of numbers.
- size is a tuple containing the number of rows and columns.
- tc is either 'i','d' or 'z' for 'integer', 'real/double' and 'complex' matrices respectively.

In [2]:
a_cvx = cvx.matrix([[1,2,3],[4,5,6]])
print(a_cvx)

[ 1  4]
[ 2  5]
[ 3  6]



In [3]:
a_np = np.array([[1,2,3],[4,5,6]])
print(a_np)

[[1 2 3]
 [4 5 6]]


In numpy the dimension of the matrix is given by ```a_np.shape```

In [4]:
a_np.shape

(2, 3)

In cvxopt the dimension of the matrix is given by ```a_cvx.size```

In [5]:
a_cvx.size

(3, 2)

In the above example ```a_cvx``` is a cvxopt matrix whereas ```a_np``` is a numpy matrix. As we can see, in the cvxopt matrix if size is not provided, by default it creates the matrix column wise whereas numpy creates the matrix rowwise. 

Since we are used to working with numpy matrices I think it's best to create the matrices as numpy matrices. first and then convert them to cvxopt matrices. We can do that as:

In [6]:
b_np = np.array([[1,2,3],[4,5,6]])
print("The numpy matrix is:\n",b_np)
b_cvx = cvx.matrix(b_np)
print("The cvxopt matrix is:\n",b_cvx)

The numpy matrix is:
 [[1 2 3]
 [4 5 6]]
The cvxopt matrix is:
 [ 1  2  3]
[ 4  5  6]



In [7]:
b_np.shape == b_cvx.size

True

As we can see both the matrices are now of the same dimension even though we created the cvxopt matrix from the numpy matrix. 

**Now that we have understood cvxopt matrices, let us see how to solve a simple quadratic programming problem using cvxopt.**

## CVXOPT Quadratic Solver

In the quqdratic form the cost function will be in the form of:
$$
(\dfrac{1}{2})(ax^2 + 2bxy + cy^2) + q_1x + q_2y
$$

In the matrix form, the cost function is written as:
$$(\dfrac{1}{2})x^TPx+ q^Tx$$

Now taking the example from the cvxopt documentation:
$$
\begin{array}{ll}
\mbox{minimize}  &  2x_1^2 + x_2^2 + x_1 x_2 + x_1 + x_2 \\
\mbox{subject to} & x_1 \geq 0 \\
       & x_2 \geq 0 \\
       & x_1  + x_2  = 1
\end{array}
$$

Writing the problem in the matrix form, we get:
$$P=2*\begin{bmatrix}2 & 0.5 \\
0.5 & 1 \end{bmatrix}\text{ and }q = \begin{bmatrix}1\\1\end{bmatrix}\text{ and }G = \begin{bmatrix}-1 & 0 \\ 0 & -1\end{bmatrix}\text{ and }h = \begin{bmatrix}0 \\ 0 \end{bmatrix}\text{ and }A=\begin{bmatrix}1 & 1\end{bmatrix}\text{ and }b = \begin{bmatrix}1\end{bmatrix}$$
We multiply $2$ with the $P$ matrix so that it cancels out the $1/2$.

In [8]:
Q = 2*cvx.matrix(np.array([ [2, .5], [.5, 1] ]))
p = cvx.matrix(np.array([[1.0], [1.0]]))
G = cvx.matrix(np.array([[-1.0,0.0],[0.0,-1.0]]))
h = cvx.matrix(np.array([[0.0],[0.0]]))
A = cvx.matrix(np.array([[1.0, 1.0]]))
b = cvx.matrix(np.array([1.0]))

In [11]:
sol=cvx.solvers.qp(Q, p, G, h, A, b)

     pcost       dcost       gap    pres   dres
 0:  1.8889e+00  7.7778e-01  1e+00  3e-16  2e+00
 1:  1.8769e+00  1.8320e+00  4e-02  2e-16  6e-02
 2:  1.8750e+00  1.8739e+00  1e-03  2e-16  5e-04
 3:  1.8750e+00  1.8750e+00  1e-05  6e-17  5e-06
 4:  1.8750e+00  1.8750e+00  1e-07  2e-16  5e-08
Optimal solution found.


In [12]:
print(sol['x']) # sol['x'] contains the optimal solution

[ 2.50e-01]
[ 7.50e-01]



One of the cons of cvxopt is that we need to define the problems in the exact form required by solvers. Writing the cost function this way can sometimes be a bit complicated. However because cvxopt takes the problem in the standard form it doesn't spend any time converting the given problem to another form and therefore is much faster in finding the solutions.

*For MPC we need to write the cost function $\mid \mid X_{curr} - X_{goal} \mid \mid_2$ in the quadratic form as $(1/2)x^TPx + q^Tx$ and then simply pass the matrices to the solver to get the optimal controls.*